In [2]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
    
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

    
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [ ]:
source = "https://www.vacourts.gov/forms/home.html"
BASE = "https://www.vacourts.gov"

soup = connect_to_page(source)
content = soup.find("div", id = "content")
htmls = content.find_all("a", href = True)
htmls_ref = []
groups = []


for i in range(len(htmls)):
    if ".html" in htmls[i]['href']:
        htmls_ref.append(htmls[i]['href'])
        groups.append(htmls[i].text)
       
del htmls_ref[0]
del htmls_ref[-1]
del groups[0]
del groups[-1]
print(htmls_ref)
print(groups)

In [ ]:
subcats = []
subcats_names = []

       

for ref in range(len(htmls_ref)):
    time.sleep(2)
    soup = connect_to_page(BASE+htmls_ref[ref])
    div_content = soup.find("div", id = "content")
    group = groups[ref]
    refs = div_content.find_all("a", href= True)
    print(group)
    for i in div_content.find_all("a", href = True):
        if ".pdf" in i['href']:
            if i['href'][0]!="/":
                print(BASE+"/"+i['href'])
            else:
                print(BASE + i['href'], "------- in very top")
            # download
        elif ".html" in i['href'] and "/forms" in i['href']:
            if "http:" not in i['href']:
                sub_url = BASE+i['href']
                sub_group = i.text
                print(sub_group, htmls_ref[ref], "------------SUB URL------------")
                sub_soup = connect_to_page(sub_url)
                sub_div_content = sub_soup.find("div", id = "content")
                sub_refs = sub_div_content.find_all("a", href = True)
                for sub_ref in  sub_refs:
                    if ".pdf" in sub_ref['href'] and "inst" not in sub_ref['href']:
                        # if "/forms" not in sub_ref['href']:
                        if sub_ref['href'][0]!="/" and "http" not in sub_ref['href']:
                            temp_url = sub_url.split("/")
                            temp_url.pop()
                            if sub_ref['href'][0] !="/":
                                temp_url = "/".join(temp_url) + "/"
                            else:
                                temp_url = "/".join(temp_url)
                            print(temp_url + sub_ref['href'], "-------in top")
                        if "http" in sub_ref['href']:
                            print(sub_ref['href'], "------------in top WITH HTTP")
                        
                        else:
                            if "http" not in sub_ref['href']:
                                if sub_ref['href'][0]!="/":
                                    print(BASE + "/" + sub_ref['href'], "------------in bot WITHOUT HTTP")
                                else:
                                    print(BASE + sub_ref['href'], "------------in bot WITHOUT HTTP")  #some have entire url, not only subcomponent
                            else:
                                 print(sub_ref['href'], "------------in bot WITH HTTP")
                        # visit


In [ ]:
pdfs = {}
form_dest = "All_forms/"
jur = 'VA'
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","error"])
today = date.today().strftime("%Y-%m-%d")


for ref in range(len(htmls_ref)):
    soup = connect_to_page(BASE+htmls_ref[ref])
    div_content = soup.find("div", id = "content")
    group = groups[ref]
    refs = div_content.find_all("a", href= True)
    print(group)
    for i in refs:
        if ".pdf" in i['href'] and "inst" not in i['href']:

            if i['href'][0]!="/" and "http" not in i['href']:
                temp_url = url.split("/")
                temp_url.pop()
                temp_url = "/".join(temp_url) + "/"
                # print(temp_url)
                url = temp_url+i['href']
                print(url)
            elif "http" not in i['href']:
                url = BASE + i['href']
            else:
                url = i['href']
            filename = url.split("/")[-1]
            if filename not in pdfs:
                if group == "Mediation":
                    sibling = i.findPreviousSibling("span")
                    if sibling!=None:
                        title = sibling.text
                        title = re.sub(r"\([^()]*\)","",title)
                        title = re.sub(r"\s+", " ", title)
                        title = title.split("[")[0]
                    elif i.findPreviousSibling("strong")!=None:
                        sibling = i.findPreviousSibling("strong")
                        title = sibling.text
                        title = re.sub(r"\([^()]*\)","",title)
                        title = re.sub(r"\s+", " ", title)
                        title = title.split("[")[0]
                    else:
                        title = i.text
                        title = re.sub(r"\([^()]*\)","",title)
                        title = re.sub(r"\s+", " ", title)
                        title = title.split("[")[0]
                else:
                    title = re.sub(r"\s+", " ", i.text)
                    title = re.sub(r"\([^()]*\)","",title)
                pdfs[filename] = title
                file_id = hashme(url)
                print(url)
                print(filename)
                print(title)
                print(file_id)
                error = 0
                status = download_pdf(url,form_dest,file_id+".pdf")

                if status=="error":
                    print("###########################")
                    print("ERROR", i['href'])
                    print("###########################")
                    error = 1
                    files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,group,url,filename,today,error]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "error"]))
                else:
                    files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,group,url,filename,today,error]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "error"]))
               


        #PDFs inside subcategories
        if ".html" in i['href'] and "/forms" in i['href'] and "mediation" not in i['href']:
            if "http" not in  i['href']:
                sub_url = BASE + i['href']
            else:
                sub_url = i['href']
            print(i.text)
            sub_soup = connect_to_page(sub_url)
            # #download
            sub_div_content = sub_soup.find("div", id = "content")
            if sub_div_content!=None:
                sub_group = i.text
                sub_refs = sub_div_content.find_all("a", href= True)
                for k in sub_refs:
                    if ".pdf" in k['href'] and "inst" not in k['href']:
                        if k['href'][0]!="/" and "http" not in k['href']:
                            temp_url = sub_url.split("/")
                            temp_url.pop()
                            temp_url = "/".join(temp_url) + "/"
                            # print(temp_url)
                            sub_url = temp_url+k['href']
                            print(sub_url)
                            
                        elif "http" not in k['href']:
                            sub_url = BASE + k['href']
                        else:
                            sub_url = k['href']
                        filename = sub_url.split("/")[-1]
                        if filename not in pdfs:
                            
                            title = re.sub(r"\s+", " ", k.text)
                            title = re.sub(r"\([^()]*\)","",title)
                            pdfs[filename] = title
                            file_id = hashme(sub_url)
                            print(sub_url)
                            print(filename)
                            print(title)
                            print(file_id)
                            error = 0
                            status = download_pdf(sub_url,form_dest,file_id+".pdf")
                            if status=="error":
                                print("###########################")
                                print("ERROR", k['href'])
                                print("###########################")
                                error = 1
                                files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,sub_group,sub_url,filename,today,error]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "error"]))
                            else:
                                files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,sub_group,sub_url,filename,today,error]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "error"]))
                            
files_df.to_csv("VA_metaData.csv",index=False, encoding="utf-8")

                            